<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# uncomment on google colab, comment on kaggle.

In [38]:
!mkdir ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shusrith/betterest-preproc

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Saving kaggle.json to kaggle (1).json
betterest-preproc.zip: Skipping, found more recently modified local copy (use --force to force download)


In [39]:
!unzip /content/betterest-preproc.zip

Archive:  /content/betterest-preproc.zip
replace test/MildDemented0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [54]:
import os
import numpy as np
directory = '/content/train'

files = [file for file in os.listdir(directory) if file.endswith('.jpg')]
len(files)

10240

In [55]:
mildDem = [i for i in files if i.startswith("milDem")]
moderateDem = [i for i in files if "moderateDim" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "veryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

2560 2560 2560 2560


In [56]:
import pandas as pd
l = [[mildDem, "0"], [moderateDem, "1"], [nonDem, "2"], [veryMild, "3"]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
0    2560
1    2560
2    2560
3    2560
Name: count, dtype: int64

In [70]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(df["Path"], df["Class"], test_size=0.2, random_state=42)

In [96]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

train_df = pd.DataFrame({
    'filename': train_images,
    'class': train_labels
})
val_df = pd.DataFrame({
    'filename': test_images,
    'class': test_labels
})

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1.0/255.0)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='filename',
    y_col='class',
    target_size=(130, 110),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=None,
    x_col='filename',
    y_col='class',
    target_size=(130, 110),
    batch_size=32,
    class_mode='categorical'
)

Found 8192 validated image filenames belonging to 4 classes.
Found 2048 validated image filenames belonging to 4 classes.


In [97]:
# s = x / len(files)
# class_weights = {i:j for i, j in zip(s.index, s.values)}

# class_weights
# Optional normalization (weights sum to 1)
# total_weight = sum(class_weights.values())
# class_weights = {class_label: weight / total_weight for class_label, weight in class_weights.items()}


In [98]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np

def conv_block(filters, act='relu'):
    """Defining a Convolutional NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D((2, 2)))

    return block

In [99]:
def dense_block(units, dropout_rate, act='relu'):
    """Defining a Dense NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Dense(units, activation=act))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))

    return block

In [100]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, **kwargs):
        super(F1Score, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_states()
        self.recall.reset_states()

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           F1Score(num_classes=4)]

In [101]:
def construct_model(act='relu'):
    """Constructing a Sequential CNN architecture for performing the classification task. """

    model = Sequential([
        Input(shape=(130, 110, 3)),
#         conv_block(16),
        conv_block(32),
        conv_block(64),
        conv_block(128),
        Dropout(0.2),
        conv_block(256),
        Dropout(0.2),
        Flatten(),
        dense_block(512, 0.7),
        dense_block(256, 0.5),
        dense_block(64, 0.3),
        Dense(4, activation='softmax')
    ], name = "cnn_model")

    return model

In [102]:
import tensorflow as tf

model = construct_model()

def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -tf.reduce_sum(alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(pt_1)) - tf.reduce_sum((1 - alpha) * tf.pow(pt_0, gamma) * tf.math.log(1. - pt_0))
    return focal_loss_fixed

model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=.25), metrics=METRICS)

model.summary()



Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_91 (Sequential)  (None, 65, 55, 32)        10272     
                                                                 
 sequential_92 (Sequential)  (None, 32, 27, 64)        55680     
                                                                 
 sequential_93 (Sequential)  (None, 16, 13, 128)       221952    
                                                                 
 dropout_65 (Dropout)        (None, 16, 13, 128)       0         
                                                                 
 sequential_94 (Sequential)  (None, 8, 6, 256)         886272    
                                                                 
 dropout_66 (Dropout)        (None, 8, 6, 256)         0         
                                                                 
 flatten_13 (Flatten)        (None, 12288)             0 

In [103]:
from keras.optimizers import Adam
def lr_schedule(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        if epoch % 2 == 0:
          return learning_rate * 0.95
        else:
          return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

In [104]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_acc') > 0.995:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True

my_callback = MyCallback()

CALLBACKS = [my_callback, lr_scheduler]

In [114]:
EPOCHS = 50

history = model.fit(train_generator, validation_data=(validation_generator), callbacks=CALLBACKS, epochs=EPOCHS)


Epoch 1: LearningRateScheduler setting learning rate to 0.0004632911295630038.
Epoch 1/50
256/256 [==============================] - 55s 214ms/step - loss: 1.8497 - acc: 0.8942 - auc: 0.9855 - f1_score_5: 0.8550 - val_loss: 4.5257 - val_acc: 0.7559 - val_auc: 0.9482 - val_f1_score_5: 0.7541 - lr: 4.6329e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0004632911295630038.
Epoch 2/50
256/256 [==============================] - 56s 219ms/step - loss: 1.9289 - acc: 0.8848 - auc: 0.9840 - f1_score_5: 0.8546 - val_loss: 2.3701 - val_acc: 0.8521 - val_auc: 0.9789 - val_f1_score_5: 0.8377 - lr: 4.6329e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0004632911295630038.
Epoch 3/50
256/256 [==============================] - 54s 211ms/step - loss: 1.7734 - acc: 0.8922 - auc: 0.9866 - f1_score_5: 0.8673 - val_loss: 1.9974 - val_acc: 0.8667 - val_auc: 0.9838 - val_f1_score_5: 0.8483 - lr: 4.6329e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.00046329112

In [ ]:
x, y = next(train_generator)
print('Train data:', x.shape, y.shape)

x, y = next(validation_generator)
print('Validation data:', x.shape, y.shape)

In [106]:
import os
import numpy as np
directory = '/content/test'

files = [file for file in os.listdir(directory) if file.endswith('.jpg')]
len(files)

1279

In [107]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [108]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [109]:
from keras.preprocessing.image import load_img, img_to_array
img = [load_img(i, target_size=(130, 110)) for i in df["Path"]]
img = [img_to_array(im) for im in img]
img = [im / 255.0 for im in img]
img = np.stack(img, axis=0)


In [110]:
labels = df["Class"]
labels = to_categorical(labels, num_classes=4)
print(labels.shape)

(1279, 4)


In [111]:
l = []
for i in labels:
  for j in range(4):
    if i[j] == 1:
      l.append(j)


In [115]:
y_prob = model.predict(img)
y_prob = np.argmax(y_prob, axis=-1)

40/40 [==============================] - 1s 15ms/step


In [116]:
from sklearn.metrics import classification_report

report = classification_report(l, y_prob, output_dict=True)
for i in report:
  print(i, report[i])

0 {'precision': 0.5, 'recall': 0.6201117318435754, 'f1-score': 0.5536159600997506, 'support': 179}
1 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}
2 {'precision': 0.8628048780487805, 'recall': 0.4421875, 'f1-score': 0.5847107438016529, 'support': 640}
3 {'precision': 0.4773662551440329, 'recall': 0.7767857142857143, 'f1-score': 0.5913338997451147, 'support': 448}
accuracy 0.5801407349491791
macro avg {'precision': 0.46004278329820336, 'recall': 0.45977123653232244, 'f1-score': 0.4324151509116296, 'support': 1279}
weighted avg {'precision': 0.6689251010600049, 'recall': 0.5801407349491791, 'f1-score': 0.5771929006854767, 'support': 1279}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.evaluate(img, labels)